In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
from core.dprocessing import *
from core.optimizers import *
from core.utils import *
from core.objects import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [4]:
df = pd.read_csv('C:/Users/minha/Research/hoffmanlab/featuresel/data/untuned_model_feature_importance.csv')
feature_importance = df.values.tolist()

In [5]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)
labels = dataset.iloc[:, [984]]

In [6]:
features1 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps', 'derivatives']
timestep1 = [98, 98, 98, 25, 98, 15, 96] 

## testing new optimizer object for efficiency etc

In [ ]:
model = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False)

In [ ]:
optim = Optimizer(model, features1, timestep1, feature_importance, labels, dataset, ligands)

In [ ]:
optim.model

In [ ]:
optim.fit(bound=0.4)

In [ ]:
optim.model

In [ ]:
new_feature_importances = optim.model.feature_importances_.argsort()

In [ ]:
new_feature_importances = new_feature_importances[::-1]

In [ ]:
nfi_names = optim.df.columns[new_feature_importances]
# need to turn into list of lists bc functions are kinda trash rn

In [ ]:
nfi_names = pd.DataFrame(nfi_names)

In [ ]:
model2 = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False)

In [ ]:
optim2 = Optimizer(model2, features1, timestep1, nfi_names.values, labels, dataset, ligands)

In [ ]:
optim2.fit(bound=0.75) # bound does not necessarily have to be 0.4

### gridsearching now

#### below parameters are just some params i found online, relatively arbitrary (i think)

In [34]:
gridsearchmodel = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False, learning_rate=0.1, n_estimators=1000)

In [35]:
optim3 = Optimizer(gridsearchmodel, features1, timestep1, feature_importance, labels, dataset, ligands)

In [36]:
optim3.fit(bound=0.4)

[20:07:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.51      0.57      0.54       196
         FLA       0.56      0.52      0.54       143
         FSL       0.59      0.62      0.60       189
         LPS       0.63      0.74      0.68       149
         P3K       0.64      0.52      0.58       185
         PIC       0.77      0.72      0.74       165
         R84       0.81      0.85      0.83       249
         TNF       0.74      0.67      0.70       162

    accuracy                           0.66      1438
   macro avg       0.66      0.65      0.65      1438
weighted avg       0.66      0.66      0.66      1438



In [ ]:
# slight improvments with these params, need to gridsearch tune

In [7]:
data = dataset.iloc[:, [i for i in range(984)]]

In [8]:
labels = dataset.iloc[:, [984]]

In [9]:
datan = data.to_numpy()
labelsn = labels.to_numpy()

In [10]:
labelsn = labelsn.reshape((-1, ))

In [11]:
partitioned_data = partition_features(features1, timestep1, feature_importance, 0.4, dataset)

In [12]:
X = partitioned_data.to_numpy()
Y = labelsn

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(datan, labelsn, test_size=0.1, random_state=42)

In [13]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [21]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
random_search = RandomizedSearchCV(gridsearchmodel, param_distributions=params, n_iter=param_comb, n_jobs=1, cv=skf.split(X, Y), 
                                   verbose=3, random_state=42)

# grid search will not output anything if you use more than 1 thread

In [22]:
# random_search = GridSearchCV(estimator=gridsearchmodel, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X, Y), verbose=1)

### Brute force gridsearch

In [23]:
random_search.fit(X, Y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[20:00:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/3] END colsample_bytree=0.6, gamma=1.5, max_depth=4, min_child_weight=10, subsample=0.8;, score=0.663 total time=  13.8s
[20:00:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/3] END colsample_bytree=0.6, gamma=1.5, max_depth=4, min_child_weight=10, subsample=0.8;, score=0.660 total time=  13.8s
[20:00:28] WARNING: C:/Users/Administrator/workspace/xgboost-win

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x00000208AFE78E40>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.1,
                                           max_delta_step=None, max_depth=None,
                                           min...
                                           reg_alpha=None, reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None,
                     

In [24]:
random_search.best_params_

{'subsample': 0.6,
 'min_child_weight': 10,
 'max_depth': 4,
 'gamma': 1,
 'colsample_bytree': 0.6}

In [39]:
gridsearchmodel

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

### trying new params from random grid search to see improvements (probably very slight)

In [27]:
rgs_model = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False, learning_rate=0.1, n_estimators=1000,
                              subsample=0.6, min_child_weight=10, max_depth=4, gamma=1, colsample_bytree=0.6)

In [29]:
rgs_model.fit(X_train, y_train)

[20:05:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [31]:
pred = rgs_model.predict(X_val)

In [32]:
cr = classification_report(y_val, pred, target_names=ligands)

In [33]:
print(cr)

              precision    recall  f1-score   support

         CpG       0.52      0.56      0.54       196
         FLA       0.60      0.48      0.53       143
         FSL       0.58      0.60      0.59       189
         LPS       0.61      0.77      0.68       149
         P3K       0.65      0.54      0.59       185
         PIC       0.76      0.73      0.74       165
         R84       0.78      0.84      0.81       249
         TNF       0.72      0.67      0.69       162

    accuracy                           0.66      1438
   macro avg       0.65      0.65      0.65      1438
weighted avg       0.66      0.66      0.65      1438



# BRUTE FORCE GRID SEARCH

In [41]:
folds = 3
# param_comb = 5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
bruteforce = GridSearchCV(gridsearchmodel, param_grid=params, n_jobs=1, cv=skf.split(X, Y), 
                                   verbose=50)

# grid search will not output anything if you use more than 1 thread

In [ ]:
bruteforce.fit(X, Y)

Fitting 3 folds for each of 405 candidates, totalling 1215 fits
[CV 1/3; 1/405] START colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6
[20:10:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/3; 1/405] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.664 total time=  16.7s
[CV 2/3; 1/405] START colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6
[20:11:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to res